# INTDB Profile Mapping

    Query current INTDB profiles. Need rate classes & profiles to compare with ESG & Intellometry. Process enhancements needed for rate class parsing, filling in missing profiles, tags, etc. etc.

    How to best use ESG, Intellometry & EWX as datasources?

## Import Modules

    use requirements.txt instead
    set venv on server?

In [146]:
import numpy as np
import datetime as dt
import pandas as pd
import os
import time
import math
import json
import cx_Oracle
base = os.getcwd()

## DB Connection

In [147]:
''' connect to TPINT (or TPPE, LPSS) '''
''' parameters: {'query' : 'str of SQL query'} '''
''' output: {'result_list' : 'list of results returned from query'} '''

def OracleAPI(query):
    
    uid = 'tesi_interface'
    pwd = 'intint88'

    ip = '172.25.152.12'
    port = '1737'
    service_name = 'tpint.mytna.com'
    #dsn = cx_Oracle.makedsn(ip, port, service_name=service_name)
    
    print('querying...')
    
    con = cx_Oracle.connect(uid, pwd, service_name)
    cur = con.cursor()
    cur.execute(query)
    columns = [i[0] for i in cur.description]

    result_list = []
    
    for result in cur:
        result_list.append(result)
        i = len(result_list)
        if (i > 0 and i % 1000 == 0):
            print('done with {}.'.format(i))
    
    print('finished with {} results, outputting dataframe.'. format(len(result_list)))
    result = pd.DataFrame(result_list)
    result.columns = columns
    
    return(result)

## INTDB Query

In [148]:
INTDB = "select * from M_ESG_PC_RC_SOURCE a, m_lpss_pc_rc b where A.SOURCEID = B.ESG_SOURCEID"

In [149]:
intdb_result = OracleAPI(INTDB)

querying...
done with 1000.
done with 2000.
done with 3000.
done with 4000.
done with 5000.
done with 6000.
done with 7000.
finished with 7581 results, outputting dataframe.


In [150]:
intdb_result.head()

,SOURCEID,MARKET,DISCO_CODE,LOAD_PROFILE,RATE_CLASS,ESG_SOURCEID,MARKET,DISCO_CODE,PROFILE_CLASS,RATE_CLASS,METER_TYPE,LOSS_CODE,LOSS_DESCRIPTION,STATE,RES_FLAG,SEQ_ID,DEFAULT_FLAG
0,10001,ERCOT,CPT,None,990,10001,ERCOT,CPT,None,ERCOT_BUS,IDR,D,SECONDARY,TX,0.0,1,N
1,10002,ERCOT,CPT,None,991,10002,ERCOT,CPT,None,ERCOT_BUS,IDR,D,SECONDARY,TX,0.0,2,N
2,10003,ERCOT,CPT,None,992,10003,ERCOT,CPT,None,ERCOT_BUS,IDR,E,PRIMARY,TX,0.0,3,N
3,10004,ERCOT,CPT,None,993,10004,ERCOT,CPT,None,ERCOT_BUS,IDR,T,TRANSMISSION,TX,0.0,4,N
4,10005,ERCOT,CPT,None,994,10005,ERCOT,CPT,None,ERCOT_BUS,IDR,D,SECONDARY,TX,0.0,5,Y


In [151]:
intdb_result.tail()

,SOURCEID,MARKET,DISCO_CODE,LOAD_PROFILE,RATE_CLASS,ESG_SOURCEID,MARKET,DISCO_CODE,PROFILE_CLASS,RATE_CLASS,METER_TYPE,LOSS_CODE,LOSS_DESCRIPTION,STATE,RES_FLAG,SEQ_ID,DEFAULT_FLAG
7576,54548,PJM,APSPA,GSCS,WP-GP30SHD,54548,PJM,APSPA,GSCS,WP-GP30SHD,SCALAR,SUBTRANSMISSION,WP_SUB,PA,0.0,9598,N
7577,54549,PJM,APSPA,GSCS,WP-GP30SHD,54549,PJM,APSPA,GSCS_IDR,WP-GP30SHD,IDR,PRIMARY,WP_PRI,PA,0.0,9599,N
7578,54550,PJM,APSPA,GSCS,WP-GP30SHD,54550,PJM,APSPA,GSCS_IDR,WP-GP30SHD,IDR,SUBTRANSMISSION,WP_SUB,PA,0.0,9600,Y
7579,54551,PJM,APSPA,GSCS,WP-GP30SHF,54551,PJM,APSPA,GSCS,WP-GP30SHF,SCALAR,SECONDARY,WP_SEC,PA,0.0,9601,Y
7580,54552,PJM,APSPA,GSCS,WP-GP30SHF,54552,PJM,APSPA,GSCS_IDR,WP-GP30SHF,IDR,SECONDARY,WP_SEC,PA,0.0,9602,Y


## INTDB Results

    Want to find missing profiles, differences between ESG and INTDB, places with large # parsing errors.

In [152]:
intdb = intdb_result.loc[:,~intdb_result.columns.duplicated()]
intdb.head()

,SOURCEID,MARKET,DISCO_CODE,LOAD_PROFILE,RATE_CLASS,ESG_SOURCEID,PROFILE_CLASS,METER_TYPE,LOSS_CODE,LOSS_DESCRIPTION,STATE,RES_FLAG,SEQ_ID,DEFAULT_FLAG
0,10001,ERCOT,CPT,None,990,10001,None,IDR,D,SECONDARY,TX,0.0,1,N
1,10002,ERCOT,CPT,None,991,10002,None,IDR,D,SECONDARY,TX,0.0,2,N
2,10003,ERCOT,CPT,None,992,10003,None,IDR,E,PRIMARY,TX,0.0,3,N
3,10004,ERCOT,CPT,None,993,10004,None,IDR,T,TRANSMISSION,TX,0.0,4,N
4,10005,ERCOT,CPT,None,994,10005,None,IDR,D,SECONDARY,TX,0.0,5,Y


In [153]:
intdb.shape

(7581, 14)

# ALPS Missing Mappings

In [154]:
alps = pd.read_csv('ALPS_PARSER_MISSING_MAPS.csv')
alps.head()

,UID_MAP,CREATED_USER,CREATED_DT,UPDATED_USER,UPDATED_DT,MARKET_CODE,DISCO_CODE,ZONE,RATE,PROFILE,...,METER_TYPE,STRATA,OUT_ZONE,OUT_RATE,OUT_PROFILE,OUT_VOLTAGE,OUT_METER_TYPE,OUT_STRATA,OUT_IGNORE_FLG,COMMENTS
0,"1,130,355",TESI_INTERFACE,08/29/2016 11:43:44,TESI_INTERFACE,06/06/2017 10:35:10,NEPOOL,COMELEC,SEMA,R-5,NaN,...,NaN,NaN,SEMA,R5,R1,SECONDARY,NaN,NaN,N,SERNA_USGH_20170601_133012_R143490112_USGH_USG...
1,"1,131,086",TESI_INTERFACE,08/30/2016 07:06:20,TESI_INTERFACE,06/04/2017 12:36:55,PJM,ATSITE,FEOHIOTE,TE-POLSD,SL,...,NaN,NaN,FEOHIOTE,TE-POLSD,SL,NaN,NaN,NaN,N,SERNA_USGH_20170604_131356_R143638625_USGH_USG...
2,"1,131,109",TESI_INTERFACE,08/30/2016 07:14:19,TESI_INTERFACE,04/20/2017 07:31:10,PJM,ATSITE,FEOHIOTE,TE-GTD,C1,...,NaN,NaN,FEOHIOTE,TE-GTD,C1,NaN,NaN,NaN,N,SERNA_USGH_20170420_074801_R140919998_USGH_USG...
3,"1,131,300",TESI_INTERFACE,08/30/2016 08:14:13,TESI_INTERFACE,03/15/2017 08:26:51,PJM,DPL,DPL,USGSSPHTG,MDDGS,...,NaN,NaN,DPL,USGSSPHTG,MDDGS,SECONDARY_120_480V,NaN,NaN,N,SERNA_USGH_20170314_091525_R138696040_USGH_USG...
4,"1,131,301",TESI_INTERFACE,08/30/2016 08:14:13,TESI_INTERFACE,06/02/2017 10:04:48,PJM,PEPCODC,PEPCODC,180,DCND,...,NaN,NaN,PEPCODC,80,DCND,GSLV,NaN,NaN,N,SERNA_USGH_20170602_101419_R143559487_USGH_USG...


In [155]:
alps

,UID_MAP,CREATED_USER,CREATED_DT,UPDATED_USER,UPDATED_DT,MARKET_CODE,DISCO_CODE,ZONE,RATE,PROFILE,...,METER_TYPE,STRATA,OUT_ZONE,OUT_RATE,OUT_PROFILE,OUT_VOLTAGE,OUT_METER_TYPE,OUT_STRATA,OUT_IGNORE_FLG,COMMENTS
0,"1,130,355",TESI_INTERFACE,08/29/2016 11:43:44,TESI_INTERFACE,06/06/2017 10:35:10,NEPOOL,COMELEC,SEMA,R-5,NaN,...,NaN,NaN,SEMA,R5,R1,SECONDARY,NaN,NaN,N,SERNA_USGH_20170601_133012_R143490112_USGH_USG...
1,"1,131,086",TESI_INTERFACE,08/30/2016 07:06:20,TESI_INTERFACE,06/04/2017 12:36:55,PJM,ATSITE,FEOHIOTE,TE-POLSD,SL,...,NaN,NaN,FEOHIOTE,TE-POLSD,SL,NaN,NaN,NaN,N,SERNA_USGH_20170604_131356_R143638625_USGH_USG...
2,"1,131,109",TESI_INTERFACE,08/30/2016 07:14:19,TESI_INTERFACE,04/20/2017 07:31:10,PJM,ATSITE,FEOHIOTE,TE-GTD,C1,...,NaN,NaN,FEOHIOTE,TE-GTD,C1,NaN,NaN,NaN,N,SERNA_USGH_20170420_074801_R140919998_USGH_USG...
3,"1,131,300",TESI_INTERFACE,08/30/2016 08:14:13,TESI_INTERFACE,03/15/2017 08:26:51,PJM,DPL,DPL,USGSSPHTG,MDDGS,...,NaN,NaN,DPL,USGSSPHTG,MDDGS,SECONDARY_120_480V,NaN,NaN,N,SERNA_USGH_20170314_091525_R138696040_USGH_USG...
4,"1,131,301",TESI_INTERFACE,08/30/2016 08:14:13,TESI_INTERFACE,06/02/2017 10:04:48,PJM,PEPCODC,PEPCODC,180,DCND,...,NaN,NaN,PEPCODC,80,DCND,GSLV,NaN,NaN,N,SERNA_USGH_20170602_101419_R143559487_USGH_USG...
5,"1,131,302",TESI_INTERFACE,08/30/2016 08:14:13,TESI_INTERFACE,04/07/2017 09:04:11,PJM,DPL,DPL,DMEL_OL,MDDOL,...,NaN,NaN,DPL,DMEL_OL,MDDOL,NaN,NaN,NaN,N,SERNA_USGH_20170407_093510_R140167172_USHI_USG...
6,"1,131,303",TESI_INTERFACE,08/30/2016 08:14:13,TESI_INTERFACE,03/04/2017 08:46:22,PJM,DPL,DPL,UGSPTOU,MDDGP,...,NaN,NaN,DPL,UGSPTOU,MDDGP,NaN,NaN,NaN,N,SERNA_USGH_20170304_090014_R138147247_USHI_USG...
7,"1,131,353",TESI_INTERFACE,08/30/2016 08:59:42,TESI_INTERFACE,05/22/2017 16:14:21,PJM,AEPOP,AEPOP,900,CCOPSLTA,...,NaN,NaN,AEPOP,900,CCOPSLTA,NaN,NaN,NaN,N,SERNA_USGH_20170522_165539_R142882940_USGH_USG...
8,"1,131,354",TESI_INTERFACE,08/30/2016 08:59:42,TESI_INTERFACE,05/22/2017 16:14:21,PJM,AEPOP,AEPOP,901,CCOPSLTA,...,NaN,NaN,AEPOP,901,CCOPSLTA,NaN,NaN,NaN,N,SERNA_USGH_20170522_165539_R142882940_USGH_USG...
9,"1,131,449",TESI_INTERFACE,08/30/2016 09:43:47,TESI_INTERFACE,03/18/2017 10:50:00,MISO,AMERENIP,AMILSZE,DS5,LITE,...,NaN,NaN,AMILSZE,DS5,LITE,NaN,NaN,NaN,N,SERNA_USGH_20170318_102438_R138978619_USHI_USG...


In [156]:
alps.shape

(1513, 21)

# Compare ALPS vs INTDB

    group data by (market, disco)

In [157]:
intdb['GROUP'] = list(zip(intdb['MARKET'], intdb['DISCO_CODE']))

c:\users\wb5888\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [158]:
alps['GROUP'] = list(zip(alps['MARKET_CODE'], alps['DISCO_CODE']))

In [159]:
intdb['GROUP'].head()

0    (ERCOT, CPT)
1    (ERCOT, CPT)
2    (ERCOT, CPT)
3    (ERCOT, CPT)
4    (ERCOT, CPT)
Name: GROUP, dtype: object

In [242]:
common = pd.merge(intdb, alps['GROUP'], how = 'inner', on = 'GROUP')
common.drop_duplicates(keep = 'first',inplace = True)
common.reset_index(drop = True, inplace = True)

In [243]:
common

,SOURCEID,MARKET,DISCO_CODE,LOAD_PROFILE,RATE_CLASS,ESG_SOURCEID,PROFILE_CLASS,METER_TYPE,LOSS_CODE,LOSS_DESCRIPTION,STATE,RES_FLAG,SEQ_ID,DEFAULT_FLAG,GROUP
0,10001,ERCOT,CPT,None,990,10001,None,IDR,D,SECONDARY,TX,0.0,1,N,"(ERCOT, CPT)"
1,10002,ERCOT,CPT,None,991,10002,None,IDR,D,SECONDARY,TX,0.0,2,N,"(ERCOT, CPT)"
2,10003,ERCOT,CPT,None,992,10003,None,IDR,E,PRIMARY,TX,0.0,3,N,"(ERCOT, CPT)"
3,10004,ERCOT,CPT,None,993,10004,None,IDR,T,TRANSMISSION,TX,0.0,4,N,"(ERCOT, CPT)"
4,10005,ERCOT,CPT,None,994,10005,None,IDR,D,SECONDARY,TX,0.0,5,Y,"(ERCOT, CPT)"
5,10006,ERCOT,CPT,None,995,10006,None,IDR,D,SECONDARY,TX,0.0,6,Y,"(ERCOT, CPT)"
6,10007,ERCOT,CPT,None,996,10007,None,IDR,E,PRIMARY,TX,0.0,7,Y,"(ERCOT, CPT)"
7,10008,ERCOT,CPT,None,997,10008,None,IDR,T,TRANSMISSION,TX,0.0,8,Y,"(ERCOT, CPT)"
8,10009,ERCOT,CPT,None,970,10009,None,IDR,D,SECONDARY,TX,0.0,9,N,"(ERCOT, CPT)"
9,10010,ERCOT,CPT,None,971,10010,None,IDR,D,SECONDARY,TX,0.0,10,N,"(ERCOT, CPT)"
